In [1]:
import pandas as pd
drive_file_path = r"C:\Users\micha\Downloads\BERKELEY Copy of Car Thefts (JL only).xlsx"

In [2]:
#for the file at the drive_file_path read the name of each sheet
sheet_names = pd.ExcelFile(drive_file_path).sheet_names
print(sheet_names)


['2023 Car Thefts', '2022 Car Thefts', '2021 Car Thefts', '2020 Car Thefts', '2019 Car Thefts', '2018 Car Thefts']


In [3]:
#for each sheet in the file at the drive_file_path, take the Date, City, and State columns and put them into a new dataframe
df = pd.DataFrame()
for sheet in sheet_names:
    #for 2023, 2022 and 2021 pull in the notes column as well
    if sheet in ['2023 Car Thefts']:
        temp_df = pd.read_excel(drive_file_path, sheet_name=sheet, usecols=["Date", "City", "State","Notes","Victim(s) (Count)"])
        temp_df.rename(columns={"Victim(s) (Count)":"Victims (JL)"}, inplace=True)
    elif sheet in ['2022 Car Thefts', '2021 Car Thefts']:
        temp_df = pd.read_excel(drive_file_path, sheet_name=sheet, usecols=["Date", "City", "State","Notes","Victims (JL)"])
    else:
        temp_df = pd.read_excel(drive_file_path, sheet_name=sheet, usecols=["Date", "City", "State","Victims (JL)"])
        temp_df["Notes"] = None
    df = pd.concat([df, temp_df])
#set nans in Victims (JL) to 0 
df["Victims (JL)"] = df["Victims (JL)"].fillna(0)
#drop any rows where any column except for the notes column is null
df = df.dropna(subset=["Date", "City", "State"])
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
#convert all City and State rows to uppercase
df["City"] = df["City"].str.upper()
df["State"] = df["State"].str.upper()
#trim the city and state columns
df["City"] = df["City"].str.strip()
df["State"] = df["State"].str.strip()
df['City'] = df['City'].str.replace('ST. LOUIS', 'SAINT LOUIS')

C:\Users\micha\AppData\Local\Temp\ipykernel_25076\2474620568.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
  df['City'] = df['City'].str.replace('ST. LOUIS', 'SAINT LOUIS')


In [4]:
#rename Victims (JL) to Victims
df.rename(columns={"Victims (JL)":"Victims"}, inplace=True)
df.dtypes

Date       datetime64[ns]
City               object
State              object
Victims           float64
Notes              object
dtype: object

In [5]:
df.head()

,Date,City,State,Victims,Notes
1,2023-01-02,PROVIDENCE,RI,2.0,Car stolen when dad went inside store. Dad for...
2,2023-01-03,TULLAHOMA,TN,1.0,Car stolen while parent went into store. Suspe...
3,2023-01-06,GASTONIA,NC,1.0,Car stolen by juvenile with underaged kid insi...
4,2023-01-08,KENT,WA,1.0,Mom asked random man to watch car while she di...
5,2023-01-08,OCEANSIDE,CA,2.0,Car stolen on street. Children dropped off unh...


In [6]:
df.tail()

,Date,City,State,Victims,Notes
154,2018-01-11,MILWAUKEE,WI,1.0,None
155,2018-04-01,MILWAUKEE,WI,1.0,None
156,2018-11-22,GREENVILLE,WI,1.0,None
157,2018-12-06,MADISON,WI,2.0,None
158,2018-04-26,RIPLEY,WV,1.0,None


In [7]:
import pandas as pd
from fuzzywuzzy import process

# Read the CSV file from the URL
df_lkpup = pd.read_csv('https://raw.githubusercontent.com/scpike/us-state-county-zip/master/geo-data.csv')
#make all the values uppercase in the city and state_abbr columns
df_lkpup['city'] = df_lkpup['city'].str.upper()
df_lkpup['state_abbr'] = df_lkpup['state_abbr'].str.upper()
df_lkpup['city_state'] = df_lkpup['city'] + df_lkpup['state_abbr']

def get_zip_code(city, state):
    # Filter the DataFrame for the given city and state
    filtered_df = df_lkpup[(df_lkpup['city'] == city) & (df_lkpup['state_abbr'] == state)]
    
    # If there are any results, return the first zip code. Otherwise, return None.
    if not filtered_df.empty:
        return filtered_df.iloc[0]['zipcode']
    else:
        # Use fuzzy matching to find the closest city and state using the city_state column
        match = process.extractOne(city+state, df_lkpup['city_state'])
        if match[1] > 90:
            # Filter the DataFrame for the matched city and state
            matched_df = df_lkpup[df_lkpup['city_state'] == match[0]]
            if not matched_df.empty:
                return matched_df.iloc[0]['zipcode']
    
    return None



In [8]:
#for each row in the df dataframe, apply the get_zip_code function to the City and State columns and put the result in a new column called Zip Code
df['Zip Code'] = df.apply(lambda row: get_zip_code(row['City'], row['State']), axis=1)

In [9]:
df.head()

,Date,City,State,Victims,Notes,Zip Code
1,2023-01-02,PROVIDENCE,RI,2.0,Car stolen when dad went inside store. Dad for...,02903
2,2023-01-03,TULLAHOMA,TN,1.0,Car stolen while parent went into store. Suspe...,None
3,2023-01-06,GASTONIA,NC,1.0,Car stolen by juvenile with underaged kid insi...,28052
4,2023-01-08,KENT,WA,1.0,Mom asked random man to watch car while she di...,98031
5,2023-01-08,OCEANSIDE,CA,2.0,Car stolen on street. Children dropped off unh...,92054


In [10]:
#nans in the Zip Code column
df_nozip = df[df['Zip Code'].isna()]
df_nozip.to_csv('no_zip.csv', index=False)

In [11]:
df_nodate = df[df['Date'].isna()]
df_nodate.to_csv('no_date.csv', index=False)

In [32]:
social_determinants = r"Y:\Yoshi\MIDS\Datasci209\Projects\FinalProject\Data\COMBINED_VVI_CENSUS_TRACT_ZIP_CODE_PUBLIC.xlsx"
#read zipcode column as string
social_determinants_df = pd.read_excel(social_determinants, sheet_name='vvi_zip_code_public', dtype={'zipcode': str})
#rename zipcode to Zip Code 
social_determinants_df = social_determinants_df.rename(columns={"zipcode": "Zip Code"})
#change to string and add leading 0s to the zip code column
social_determinants_df['Zip Code'] = social_determinants_df['Zip Code'].astype(str).str.zfill(5)
social_determinants_df.head()

,Zip Code,VVI,Economic,Education,HealthCareAccess,Neighborhood,Housing,CleanEnvironment,Social,Transportation,PublicSafety
0,00501,-0.558895,-1.418232,-0.421614,-1.297315,-1.176824,0.057077,0.195511,-0.625438,-0.303399,0.108954
1,00544,-0.558895,-1.418232,-0.421614,-1.297315,-1.176824,0.057077,0.195511,-0.625438,-0.303399,0.108954
2,01001,-0.429593,-0.530305,-0.627637,-1.270484,-0.068406,0.217659,0.756576,-0.127740,0.706350,1.083041
3,01002,-1.982873,0.061159,-2.560182,-1.072013,0.354237,1.785441,0.110258,0.207606,0.716703,0.457081
4,01003,-0.576723,1.891960,0.399121,-1.083153,2.218414,4.296579,0.368681,5.999105,-0.726850,0.397524


In [33]:
#group by zip code and take the mean of the other columns
#social_determinants_df = social_determinants_df.groupby('Zip Code').mean().reset_index()
#social_determinants_df.to_csv(r'Y:\Yoshi\MIDS\Datasci209\Projects\FinalProject\Data\social_determinants.csv', index=False)


In [34]:
#for rows where the zip code is not null merge the df dataframe with the social_determinants_df dataframe on the Zip Code column
df = df.merge(social_determinants_df, on="Zip Code", how="left")
df.head()

,Date,City,State,Victims,Notes,Zip Code,VVI,Economic,Education,HealthCareAccess,Neighborhood,Housing,CleanEnvironment,Social,Transportation,PublicSafety
0,2023-01-02,PROVIDENCE,RI,2.0,Car stolen when dad went inside store. Dad for...,02903,0.303314,0.635844,-1.364393,-1.588516,-0.947043,2.541139,1.659430,0.754843,3.462800,1.364576
1,2023-01-03,TULLAHOMA,TN,1.0,Car stolen while parent went into store. Suspe...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-06,GASTONIA,NC,1.0,Car stolen by juvenile with underaged kid insi...,28052,2.248729,0.884267,0.593603,-0.388542,1.201116,1.192807,1.533170,1.866413,0.409392,-1.177375
3,2023-01-08,KENT,WA,1.0,Mom asked random man to watch car while she di...,98031,-0.648801,-0.719367,-0.287521,-1.025243,-1.086965,0.517744,1.785542,-0.265366,-0.082540,0.660024
4,2023-01-08,OCEANSIDE,CA,2.0,Car stolen on street. Children dropped off unh...,92054,-0.875484,-0.148338,-0.476303,-0.675651,-1.198332,2.006442,0.867796,0.231651,-0.086043,1.082233


In [35]:
df.to_csv(r"C:\Users\micha\Downloads\car_theftsNew.csv", index=False)